# Smart News Analyzer & Fact Checker

**LangGraph + Gemini 2.5 Flash Demo** (No LangChain)

This notebook demonstrates:
- **LangGraph state machine** with multiple nodes and conditional edges
- **Gemini 2.5 Flash** for text analysis and claim extraction
- **Gemini Google Search tool** for real-time fact verification
- **Human-in-the-loop** interrupts for uncertain claims
- **Failure recovery** (bad URL → ask user to paste text)
- **SQLite checkpoints** for state persistence
- **LangSmith tracing** for observability in Studio

## How to run

1. Start LangGraph Studio in terminal: `langgraph dev`
2. Run all cells below
3. Use the Gradio chat interface to analyze news articles
4. Watch the graph execute in LangSmith Studio


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# Configure LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "smart-news-analyzer"

print("\n📊 LangSmith tracing enabled for project: smart-news-analyzer")



📊 LangSmith tracing enabled for project: smart-news-analyzer


In [2]:

import uuid
from smart_news_graph import get_graph_with_checkpointer, Command
from langgraph.types import Command

graph = get_graph_with_checkpointer()

# Generate a unique thread ID for this session (or reuse one for recovery)
THREAD_ID = f"session-{uuid.uuid4().hex[:8]}"
print(f"🧵 Thread ID: {THREAD_ID}")
print("   (Save this ID to resume from checkpoints after kernel restart)")

# Store pending interrupt info
pending_interrupt = None


🧵 Thread ID: session-d9d8e5e5
   (Save this ID to resume from checkpoints after kernel restart)


In [3]:
# Helper functions for running the graph with interrupt handling

def run_graph_step(user_input: str, thread_id: str = None):
    """
    Run the graph with user input. Handles interrupts gracefully.
    Returns: (response_text, is_complete, interrupt_info)
    """
    global pending_interrupt
    thread_id = thread_id or THREAD_ID
    config = {"configurable": {"thread_id": thread_id}}
    
    # Check if we're resuming from an interrupt
    if pending_interrupt:
        try:
            result = graph.invoke(Command(resume=user_input), config)
            pending_interrupt = None
        except Exception as e:
            return f"Error resuming: {str(e)}", False, None
    else:
        # Fresh run
        initial_state = {
            "messages": [{"role": "user", "content": user_input}]
        }
        try:
            result = graph.invoke(initial_state, config)
        except Exception as e:
            return f"Error: {str(e)}", False, None
    
    # Check if we hit an interrupt
    state = graph.get_state(config)
    
    if state.next:
        # Graph is paused at an interrupt
        # Look for interrupt info in the state's tasks
        for task in state.tasks:
            if hasattr(task, 'interrupts') and task.interrupts:
                interrupt_data = task.interrupts[0]
                pending_interrupt = interrupt_data.value
                return pending_interrupt.get("message", "Awaiting input..."), False, pending_interrupt
        
        pending_interrupt = {"type": "unknown", "message": f"Paused at: {state.next}"}
        return f"⏸️ Graph paused at: {state.next}. Provide input to continue.", False, pending_interrupt
    
    # Graph completed
    final_report = result.get("final_report", "No report generated")
    
    # Build summary
    summary_parts = []
    
    if result.get("article_title"):
        summary_parts.append(f"📰 **Article:** {result['article_title']}")
    
    if result.get("claims"):
        summary_parts.append(f"\n📋 **Claims extracted:** {len(result['claims'])}")
    
    if result.get("verifications"):
        summary_parts.append("\n🔍 **Verification Results:**")
        for v in result["verifications"]:
            emoji = {"verified": "✅", "false": "❌", "uncertain": "❓", "needs_review": "⚠️"}.get(v["verdict"], "•")
            summary_parts.append(f"  {emoji} {v['claim'][:60]}...")
    
    if result.get("sentiment"):
        s = result["sentiment"]
        summary_parts.append(f"\n💭 **Sentiment:** {s.get('label', 'unknown')} (score: {s.get('compound', 0):.2f})")
    
    summary_parts.append(f"\n\n---\n\n## 📊 Final Report\n\n{final_report}")
    
    return "\n".join(summary_parts), True, None


print("✅ Graph runner functions loaded")


✅ Graph runner functions loaded


## Checkpoint Recovery Demo

LangGraph uses SQLite checkpoints to persist state. If the kernel crashes or you restart:

1. Note your `THREAD_ID` from above
2. Restart kernel
3. Run cells 1-3
4. Set `THREAD_ID = "your-saved-thread-id"` 
5. Run the cell below to see saved state


In [4]:
# Checkpoint inspection - View current state from SQLite checkpoint

def inspect_checkpoint(thread_id: str = None):
    """Inspect the current checkpoint state for a thread."""
    thread_id = thread_id or THREAD_ID
    config = {"configurable": {"thread_id": thread_id}}
    
    try:
        state = graph.get_state(config)
        
        print(f"🔍 Checkpoint State for thread: {thread_id}")
        print("=" * 50)
        
        if state.values:
            print(f"\n📌 Current state values:")
            for key, value in state.values.items():
                if isinstance(value, list) and len(value) > 2:
                    print(f"   {key}: [{len(value)} items]")
                elif isinstance(value, str) and len(value) > 100:
                    print(f"   {key}: {value[:100]}...")
                else:
                    print(f"   {key}: {value}")
            
            print(f"\n⏭️ Next nodes: {state.next}")
            
            if state.next:
                print("\n⚠️ Graph is paused - waiting for input to resume")
        else:
            print("No checkpoint found for this thread.")
            
    except Exception as e:
        print(f"Error reading checkpoint: {e}")

# inspect_checkpoint()
print("💾 Run inspect_checkpoint() to view saved state")


💾 Run inspect_checkpoint() to view saved state


## Gradio Chat Interface

The chat UI below allows you to:
- **Paste a URL** to a news article (the graph will fetch and analyze it)
- **Paste article text** directly if the URL fails or you prefer
- **Respond to interrupts** when the agent needs human input
- **See the graph visualization** in LangSmith Studio (running in terminal)

### Test scenarios:
1. **Normal flow**: Paste a real news URL (e.g., from BBC, CNN, Reuters)
2. **Failure + recovery**: Paste a fake URL like `https://fake-news-site.com/article` 
3. **Human-in-the-loop**: The agent will ask for your input on uncertain claims


In [5]:
# Gradio Chat Interface
import gradio as gr

def chat_handler(message: str, history: list):
    """
    Handle chat messages. Runs the graph and handles interrupts.
    """
    global pending_interrupt
    
    if not message.strip():
        return "Please enter a news article URL or paste article text."
    
    # Run the graph step
    response, is_complete, interrupt_info = run_graph_step(message.strip())
    
    return response


def reset_session():
    """Reset the session for a fresh analysis."""
    global THREAD_ID, pending_interrupt
    import uuid
    THREAD_ID = f"session-{uuid.uuid4().hex[:8]}"
    pending_interrupt = None
    return f"🔄 Session reset. New thread ID: {THREAD_ID}"


# Create the Gradio interface
with gr.Blocks(
    title="Smart News Analyzer",
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="slate",
    )
) as demo:
    gr.Markdown("""
    # 📰 Smart News Analyzer & Fact Checker
    
    **Powered by LangGraph + Gemini 2.5 Flash**
    
    Paste a news article URL or text to analyze. The agent will:
    1. Extract key claims
    2. Verify each claim using Google Search
    3. Ask for your input on uncertain claims
    4. Generate a fact-check report
    """)
    
    with gr.Row():
        with gr.Column(scale=3):
            # FIX: Removed 'type="messages"' but we will still send dictionaries below
            chatbot = gr.Chatbot(
                height=500,
                show_label=False,
                avatar_images=(None, "https://em-content.zobj.net/source/twitter/376/robot_1f916.png"),
            )
            
            msg = gr.Textbox(
                placeholder="Paste a news article URL or text here...",
                show_label=False,
                container=False,
            )
            
            with gr.Row():
                submit_btn = gr.Button("Analyze", variant="primary")
                reset_btn = gr.Button("New Session", variant="secondary")
        
        with gr.Column(scale=1):
            gr.Markdown(f"""
            ### Session Info
            
            **Thread ID:** `{THREAD_ID}`
            
            ### Instructions
            
            1. Paste URL or article text
            2. Click "Analyze"
            3. Respond to any prompts
            4. View report when complete
            
            ### Watch in Studio
            
            Run `langgraph dev` in terminal
            to see the graph execute!
            """)
            
            status_box = gr.Textbox(
                label="Status",
                value="Ready",
                interactive=False,
            )
    
    def respond(message, chat_history):
        if not message.strip():
            return "", chat_history
        
        # 1. User message as DICTIONARY
        chat_history.append({"role": "user", "content": message})
        
        # 2. Get Response
        response_text = chat_handler(message, chat_history)
        
        # 3. Assistant message as DICTIONARY
        chat_history.append({"role": "assistant", "content": response_text})
        
        return "", chat_history
    
    # Connect inputs
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    submit_btn.click(respond, [msg, chatbot], [msg, chatbot])
    
    # Reset button
    reset_btn.click(
        lambda: ([], reset_session()),
        outputs=[chatbot, status_box]
    )

# Launch the interface
demo.launch(share=False, inline=True)

/Users/ldr0/Documents/UPM/Cloud Computing/langchainPresentation/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/_3/7nx89wtj68x77c5zlhz5fztm0000gn/T/ipykernel_17575/1270978293.py:29: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
